In [ ]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.csv(file_location, header= True, inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|female|    no|sun|dinner|   2|
|     10.34|1.66|  male|    no|sun|dinner|   3|
|     21.01| 3.5|  male|    no|sun|dinner|   3|
|     23.68|3.31|  male|    no|sun|dinner|   2|
|     24.59|3.61|female|    no|sun|dinner|   4|
|     25.29|4.71|  male|    no|sun|dinner|   4|
|      8.77| 2.0|  male|    no|sun|dinner|   2|
|     26.88|3.12|  male|    no|sun|dinner|   4|
|     15.04|1.96|  male|    no|sun|dinner|   2|
|     14.78|3.23|  male|    no|sun|dinner|   2|
|     10.27|1.71|  male|    no|sun|dinner|   2|
|     35.26| 5.0|female|    no|sun|dinner|   4|
|     15.42|1.57|  male|    no|sun|dinner|   2|
|     18.43| 3.0|  male|    no|sun|dinner|   4|
|     14.83|3.02|female|    no|sun|dinner|   2|
+----------+----+------+------+---+------+----+



In [ ]:
df.printSchema

Out[3]: <bound method DataFrame.printSchema of DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: int]>

In [ ]:
df.columns

Out[4]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

## handing categorical features
### these columns should have numerical values, changing with indexer library

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
df_py=indexer.fit(df).transform(df)
df_py.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|female|    no|sun|dinner|   2|        1.0|
|     10.34|1.66|  male|    no|sun|dinner|   3|        0.0|
|     21.01| 3.5|  male|    no|sun|dinner|   3|        0.0|
|     23.68|3.31|  male|    no|sun|dinner|   2|        0.0|
|     24.59|3.61|female|    no|sun|dinner|   4|        1.0|
|     25.29|4.71|  male|    no|sun|dinner|   4|        0.0|
|      8.77| 2.0|  male|    no|sun|dinner|   2|        0.0|
|     26.88|3.12|  male|    no|sun|dinner|   4|        0.0|
|     15.04|1.96|  male|    no|sun|dinner|   2|        0.0|
|     14.78|3.23|  male|    no|sun|dinner|   2|        0.0|
|     10.27|1.71|  male|    no|sun|dinner|   2|        0.0|
|     35.26| 5.0|female|    no|sun|dinner|   4|        1.0|
|     15.42|1.57|  male|    no|sun|dinner|   2|        0.0|
|     18.43| 3.0|  male|    no|sun|dinne

In [ ]:
indexer = StringIndexer(inputCols=["smoker","day","time"], outputCols=["smoker_indexed","day_indexed","time_indexed"])
df_py=indexer.fit(df_py).transform(df_py)
df_py.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|female|    no|sun|dinner|   2|        1.0|           0.0|        0.0|         0.0|
|     10.34|1.66|  male|    no|sun|dinner|   3|        0.0|           0.0|        0.0|         0.0|
|     21.01| 3.5|  male|    no|sun|dinner|   3|        0.0|           0.0|        0.0|         0.0|
|     23.68|3.31|  male|    no|sun|dinner|   2|        0.0|           0.0|        0.0|         0.0|
|     24.59|3.61|female|    no|sun|dinner|   4|        1.0|           0.0|        0.0|         0.0|
|     25.29|4.71|  male|    no|sun|dinner|   4|        0.0|           0.0|        0.0|         0.0|
|      8.77| 2.0|  male|    no|sun|dinner|   2|        0.0|           0.0|        0.0|         0.0|


In [ ]:
df_py.columns

Out[11]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

## combining columns together

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'], outputCol="Independent Features")

output = featureAssembler.transform(df_py)

In [ ]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|female|    no|sun|dinner|   2|        1.0|           0.0|        0.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  male|    no|sun|dinner|   3|        0.0|           0.0|        0.0|         0.0|(6,[0,1],[1.66,3.0])|
|     21.01| 3.5|  male|    no|sun|dinner|   3|        0.0|           0.0|        0.0|         0.0| (6,[0,1],[3.5,3.0])|
|     23.68|3.31|  male|    no|sun|dinner|   2|        0.0|           0.0|        0.0|         0.0|(6,[0,1],[3.31,2.0])|
|     24.59|3.61|female|    no|sun|dinner|   4|        1.0|           0.0|        0.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  male|    no|s

In [ ]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|(6,[0,1],[1.66,3.0])|
| (6,[0,1],[3.5,3.0])|
|(6,[0,1],[3.31,2.0])|
|[3.61,4.0,1.0,0.0...|
|(6,[0,1],[4.71,4.0])|
| (6,[0,1],[2.0,2.0])|
|(6,[0,1],[3.12,4.0])|
|(6,[0,1],[1.96,2.0])|
|(6,[0,1],[3.23,2.0])|
|(6,[0,1],[1.71,2.0])|
|[5.0,4.0,1.0,0.0,...|
|(6,[0,1],[1.57,2.0])|
| (6,[0,1],[3.0,4.0])|
|[3.02,2.0,1.0,0.0...|
+--------------------+



In [ ]:
final_data = output.select("Independent Features","total_bill")

In [ ]:
final_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|(6,[0,1],[1.66,3.0])|     10.34|
| (6,[0,1],[3.5,3.0])|     21.01|
|(6,[0,1],[3.31,2.0])|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|(6,[0,1],[4.71,4.0])|     25.29|
| (6,[0,1],[2.0,2.0])|      8.77|
|(6,[0,1],[3.12,4.0])|     26.88|
|(6,[0,1],[1.96,2.0])|     15.04|
|(6,[0,1],[3.23,2.0])|     14.78|
|(6,[0,1],[1.71,2.0])|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|(6,[0,1],[1.57,2.0])|     15.42|
| (6,[0,1],[3.0,4.0])|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
+--------------------+----------+



In [ ]:
from pyspark.ml.regression import LinearRegression

## Train test split

train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [ ]:
regressor.coefficients

Out[20]: DenseVector([3.148, 2.6559, 4.4845, 0.0, 0.0, 0.0])

In [ ]:
regressor.intercept

Out[21]: 2.181209206694213

In [ ]:
## Predictions
pred_results=regressor.evaluate(test_data)

In [ ]:
## final comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.66,3.0])|     10.34| 15.37458563911415|
|(6,[0,1],[1.96,2.0])|     15.04|13.663105446324602|
| (6,[0,1],[3.0,4.0])|     18.43|22.248821613405305|
|[3.02,2.0,1.0,0.0...|     14.83|21.484520195759178|
+--------------------+----------+------------------+

